In [1]:
# Create the Solver object for either NEOS or a local solver
def SetUpSolver(Model):
    if Neos:
        Solver = pyo.SolverManagerFactory('neos')   # Solver on NEOS
        if pyo.value(Model.Engine) == 'bonmin':   # Non-linear
            Model.Options = {}   # Bonmin doesn't accept command line options
        elif pyo.value(Model.Engine) == 'couenne':   # Non-linear
            Model.Options = {}   # Couenne doesn't accept command line options
        elif pyo.value(Model.Engine) == 'filmint':   # Non-linear
            Model.Options = {}   # Time limit option is unknown
        elif pyo.value(Model.Engine) == 'knitro':   # Non-linear
            Model.Options = {'mip_maxtime_real': Model.TimeLimit, 'maxtime_real': Model.TimeLimit}
        elif pyo.value(Model.Engine) == 'minlp':   # Non-linear
            Model.Options = {}   # Time limit option is unknown
        elif pyo.value(Model.Engine) == 'cbc':   # Linear
            Model.Options = {'seconds': Model.TimeLimit}
        elif pyo.value(Model.Engine) == 'minto':   # Linear
            Model.Options = {'maxtime': Model.TimeLimit}
        elif pyo.value(Model.Engine) == 'mosek':   # Linear
            Model.Options = {'timelim': Model.TimeLimit}
        elif pyo.value(Model.Engine) == 'cplex':   # Linear
            Model.Options = {'timelimit': Model.TimeLimit}
        elif pyo.value(Model.Engine) == 'octeract':   # Non-linear
            Model.Options = {'MAX_SOLVER_TIME': Model.TimeLimit, 'MILP_SOLVER': 'HIGHS'}
        else:
            Model.Options = {}
            print('Unknown NEOS solver when setting options')
    else:
        Solver = pyo.SolverFactory(pyo.value(Model.Engine))   # Local solver installed
        if pyo.value(Model.Engine) == 'bonmin':   # Non-linear
            Model.Options = {}   # Bonmin doesn't accept command line options, use bonmin.opt instead
        elif pyo.value(Model.Engine) == 'couenne':   # Non-linear
            Model.Options = {}   # Couenne doesn't accept command line options, use couenne.opt instead
        elif pyo.value(Model.Engine) == 'cbc':   # Linear
            Solver.options['seconds'] = pyo.value(Model.TimeLimit)
        elif pyo.value(Model.Engine) == 'glpk':   # Linear
            Solver.options['tmlim'] = pyo.value(Model.TimeLimit)
        elif pyo.value(Model.Engine) == 'appsi_highs':   # Linear
            Solver.options['time_limit'] = pyo.value(Model.TimeLimit)
            Solver.options['log_file'] = 'highs.log'   # Sometimes HiGHS doesn't update the console as it solves, so write log file too
        elif pyo.value(Model.Engine) == 'gurobi_direct':
            Solver.options['timelimit'] = pyo.value(Model.TimeLimit)
        else:
            print('Unknown local solver when setting options')
    
    return Solver, Model

In [2]:
# Call either NEOS or a local solver
def CallSolver(Solver, Model):
    if Neos:
        Results = Solver.solve(Model, load_solutions = LoadSolution, tee = Verbose, solver = Model.Engine, options = Model.Options)
    else:
        Results = Solver.solve(Model, load_solutions = LoadSolution, tee = Verbose)
    
    return Results, Model